# <center> Assignment 2: HR Analytics - Job Change of Data Scientists </center>

## Data

A company which is active in Big Data and Data Science wants to hire data scientists among people who successfully pass some courses which conduct by the company. Many people signup for their training. Company wants to know which of these candidates are really wants to work for the company after training or looking for a new employment because it helps to reduce the cost and time as well as the quality of training or planning the courses and categorization of candidates. Information related to demographics, education, experience are in hands from candidates signup and enrollment.

This dataset designed to understand the factors that lead a person to leave current job for HR researches too. By model(s) that uses the current credentials,demographics,experience data you will predict the probability of a candidate to look for a new job or will work for the company, as well as interpreting affected factors on employee decision.

The whole data divided to train and test . Target isn't included in test but the test target values data file is in hands for related tasks. A sample submission correspond to enrollee_id of test set provided too with columns : enrollee _id , target

Note:

- The dataset is imbalanced.
- Most features are categorical (Nominal, Ordinal, Binary), some with high cardinality.
- Missing imputation can be a part of your pipeline as well.

Ref: [HR Analytics - Job Change of Data Scientists](https://www.kaggle.com/arashnic/hr-analytics-job-change-of-data-scientists)
**Data Descriptions**:

|Features|Descriptions
|-|-|
|enrollee_id| Unique ID for candidate|
|city| City code|
|city_ development _index| Developement index of the city (scaled)|
|gender| Gender of candidate|
|relevent_experience| Relevant experience of candidate|
|enrolled_university| Type of University course enrolled if any|
|education_level| Education level of candidate|
|major_discipline|Education major discipline of candidate|
|experience| Candidate total experience in years|
|company_size| No of employees in current employer's company|
|company_type| Type of current employer|
|lastnewjob| Difference in years between previous job and current job|
|training_hours| training hours completed|
|target| 0 – Not looking for job change, 1 – Looking for a job change|

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train.csv')
data.head(5)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [3]:
data.isnull().sum()

enrollee_id                  0
city                         0
city_development_index       0
gender                    4508
relevent_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
company_size              5938
company_type              6140
last_new_job               423
training_hours               0
target                       0
dtype: int64

In [4]:
# Đánh giá tỉ lệ missing value
data.education_level.isnull().sum()

460

In [5]:
# chia bins
data['city_group_idx'] = pd.cut(data['city_development_index'], bins=4, precision=0)

In [6]:
# Nhóm city group index và relevent experience, nhận thấy mode của tất cả các khoảng là Graduate
data.groupby(['city_group_idx','relevent_experience']).education_level.agg([len, pd.Series.mode ,lambda x: (x.isnull().sum()/len(x))])

len      mode  <lambda_0>
city_group_idx relevent_experience                                
(0.4, 0.6]     Has relevent experience   379  Graduate    0.031662
               No relevent experience    211  Graduate    0.037915
(0.6, 0.7]     Has relevent experience  2660  Graduate    0.020677
               No relevent experience   1306  Graduate    0.060490
(0.7, 0.8]     Has relevent experience  1380  Graduate    0.012319
               No relevent experience    513  Graduate    0.042885
(0.8, 0.9]     Has relevent experience  9373  Graduate    0.013550
               No relevent experience   3336  Graduate    0.041966

In [7]:
# Kiểm tra xem có thực là tất cả các giá trị mode trong từng khoảng đều là Graduate không?
# Khoảng 1
data[(data.city_development_index > 0.4) & 
     (data.city_development_index <= 0.6) & 
     (data.relevent_experience == 'Has relevent experience')].education_level.mode()

0    Graduate
dtype: object

In [8]:
# Khoảng 2
data[(data.city_development_index > 0.6) & 
     (data.city_development_index <= 0.7) & 
     (data.relevent_experience == 'Has relevent experience')].education_level.mode()
# Kiểm tra cho đến hết ...

0    Graduate
dtype: object

In [9]:
data.education_level.mode()[0]

'Graduate'

In [10]:
# Dùng cách groupby
ser = data.groupby(['city_group_idx','relevent_experience']).education_level.apply(lambda x: x.fillna(x.mode()[0]))
ser.isnull().sum()

0

In [11]:
# Trong trường hợp cùng 1 giá trị mode, có thể sử dụng fillna cho cả cột
ser1 = data.education_level.fillna(data.education_level.mode()[0])
ser1.isnull().sum()

0